In [22]:
library(tidyverse, quietly = TRUE)
library(psych, quietly = TRUE)
library(Hmisc, quietly = TRUE)
library(car)

In [2]:
data <- read.csv("../data/combined_data.csv")
head(data, 1)

,NAME,GEOID,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms
,<chr>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,"Census Tract 9601, Asotin County, Washington",53003960100,1569,145,1337,73942,12,58,110.3274,41.7821,51.70445


In [3]:
data <- data %>% dplyr::select(-NAME, -GEOID, -mean_u_mbps)
scaled_data <- scale(data)
head(data)

,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_lat_ms
,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1569,145,1337,73942,12,58.00000,110.3274,51.70445
2,1780,421,1352,72988,13,60.07692,145.7848,24.62630
3,1156,6,396,38077,11,60.00000,173.4079,29.98058
4,805,29,647,38777,8,62.50000,127.6005,22.16379
5,1198,26,635,42639,8,62.50000,192.4299,21.66087
6,1392,115,909,53947,9,58.88889,151.0552,26.18557


In [4]:
KMO(data)

Kaiser-Meyer-Olkin factor adequacy
Call: KMO(r = data)
Overall MSA =  0.64
MSA for each item = 
      with_internet      work_from_home sixtyfive_and_older       median_income 
               0.59                0.76                0.59                0.64 
   number_providers    mean_lowest_cost         mean_d_mbps         mean_lat_ms 
               0.81                0.69                0.60                0.63 

In [5]:
pca_data <- prcomp(data, scale = TRUE, center = TRUE)

# disable scientific notation
options(scipen = 999)
eigenvectors <- -pca_data$rotation
eigenvalues <- pca_data$sdev^2

index_col <- as.data.frame(-pca_data$x) %>%
    dplyr::select(PC1) %>%
    rename(index = PC1)

index_data <- bind_cols(scaled_data, index_col)
head(index_data)

,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_lat_ms,index
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.5167430,0.1292741,2.23493253,0.07418486,1.7094218,-0.18458632,-1.05913969,1.17991288,-1.42192033
2,0.9194434,2.9303783,2.28240921,0.07416551,2.2108891,0.41923513,-0.64056563,-0.15416598,0.02864058
3,-0.2714809,-1.2814270,-0.74343819,0.07345717,1.2079545,0.39687137,-0.31447576,0.10962741,-1.02857117
4,-0.9413759,-1.0480017,0.05100501,0.07347137,-0.2964475,1.12369349,-0.85523159,-0.27548794,-0.40670926
5,-0.1913226,-1.0784484,0.01302366,0.07354973,-0.2964475,1.12369349,-0.08992031,-0.30026584,0.29898986
6,0.1789328,-0.1751938,0.88026444,0.07377917,0.2050198,0.07383932,-0.57834859,-0.07734433,-0.17626727


In [6]:
as.list(rcorr(as.matrix(index_data), type = "pearson")$P[, ncol(index_data)])

$with_internet
[1] 0

$work_from_home
[1] 0.001261931

$sixtyfive_and_older
[1] 0.002262412

$median_income
[1] 0.9183505

$number_providers
[1] 0

$mean_lowest_cost
[1] 0.000000002657749

$mean_d_mbps
[1] 0

$mean_lat_ms
[1] 0

$index
[1] NA

In [7]:
as.list(rcorr(as.matrix(index_data), type = "pearson"))$P

,with_internet,work_from_home,sixtyfive_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_lat_ms,index
with_internet,NA,0.0000000002997904,0.00000000000000,0.03006990,0.082203264746717686,0.065587150157389,0.000001719724394889,0.0012541075067722,0.000000000000000
work_from_home,0.0000000002997904,NA,0.00000004534735,0.62244972,0.692796119793771537,0.933508191298156,0.672635578892659680,0.9085912323262457,0.001261930798470
sixtyfive_and_older,0.0000000000000000,0.0000000453473543,NA,0.04477147,0.476702946467304045,0.497188268940468,0.569411922064836595,0.1693793288053422,0.002262411882200
median_income,0.0300698982044132,0.6224497181702566,0.04477147246482,NA,0.767570871280282230,0.469668921787601,0.521089097196837159,0.4313383103711321,0.918350528428636
number_providers,0.0822032647467177,0.6927961197937715,0.47670294646730,0.76757087,NA,0.000005261422731,0.000000000006029399,0.0000000002296336,0.000000000000000
mean_lowest_cost,0.0655871501573890,0.9335081912981564,0.49718826894047,0.46966892,0.000005261422731273,NA,0.007630242115620689,0.0141555670114635,0.000000002657749
mean_d_mbps,0.0000017197243949,0.6726355788926597,0.56941192206484,0.52108910,0.000000000006029399,0.007630242115621,NA,0.0000000000000000,0.000000000000000
mean_lat_ms,0.0012541075067722,0.9085912323262457,0.16937932880534,0.43133831,0.000000000229633645,0.014155567011463,0.000000000000000000,NA,0.000000000000000
index,0.0000000000000000,0.0012619307984705,0.00226241188220,0.91835053,0.000000000000000000,0.000000002657749,0.000000000000000000,0.0000000000000000,NA


In [8]:
data1.2 <- data %>% dplyr::select(-median_income)
scaled_data1.2 <- scale(data1.2)

pca_data1.2 <- prcomp(data1.2, scale = TRUE, center = TRUE)
eigenvectors1.2 <- -pca_data1.2$rotation
eigenvalues1.2 <- pca_data1.2$sdev^2

index_col1.2 <- as.data.frame(-pca_data1.2$x) %>%
    dplyr::select(PC1) %>%
    rename(index = PC1)

index_data1.2 <- bind_cols(scaled_data1.2, index_col1.2)

In [9]:
library(leaps)
subset <- regsubsets(index ~ ., index_data1.2)
summary(subset)

Warning message in log(vr):
“NaNs produced”


Subset selection object
Call: regsubsets.formula(index ~ ., index_data1.2)
7 Variables  (and intercept)
                    Forced in Forced out
with_internet           FALSE      FALSE
work_from_home          FALSE      FALSE
sixtyfive_and_older     FALSE      FALSE
number_providers        FALSE      FALSE
mean_lowest_cost        FALSE      FALSE
mean_d_mbps             FALSE      FALSE
mean_lat_ms             FALSE      FALSE
1 subsets of each size up to 7
Selection Algorithm: exhaustive
         with_internet work_from_home sixtyfive_and_older number_providers
1  ( 1 ) " "           " "            " "                 " "             
2  ( 1 ) "*"           " "            " "                 " "             
3  ( 1 ) "*"           " "            " "                 "*"             
4  ( 1 ) "*"           " "            " "                 "*"             
5  ( 1 ) "*"           " "            " "                 "*"             
6  ( 1 ) "*"           "*"            " "              

In [16]:
forward <- regsubsets(index ~ ., index_data1.2, method = "forward")

results <- summary(forward)

results

Subset selection object
Call: regsubsets.formula(index ~ ., index_data1.2, method = "forward")
7 Variables  (and intercept)
                    Forced in Forced out
with_internet           FALSE      FALSE
work_from_home          FALSE      FALSE
sixtyfive_and_older     FALSE      FALSE
number_providers        FALSE      FALSE
mean_lowest_cost        FALSE      FALSE
mean_d_mbps             FALSE      FALSE
mean_lat_ms             FALSE      FALSE
1 subsets of each size up to 7
Selection Algorithm: forward
         with_internet work_from_home sixtyfive_and_older number_providers
1  ( 1 ) " "           " "            " "                 " "             
2  ( 1 ) "*"           " "            " "                 " "             
3  ( 1 ) "*"           " "            " "                 "*"             
4  ( 1 ) "*"           " "            " "                 "*"             
5  ( 1 ) "*"           " "            " "                 "*"             
6  ( 1 ) "*"           "*"            

In [17]:
which.max(results$adjr2)
which.min(results$bic)
which.max(results$rsq)
which.min(results$cp)

[1] 7

[1] 7

[1] 7

[1] 7

In [18]:
coefficients <- coef(forward, 7)
as.list(coefficients[which(coefficients < .05)])

$`(Intercept)`
[1] 0.00000000000000005247539

$number_providers
[1] -0.4186472

$mean_lat_ms
[1] -0.51717

In [19]:
as.list(coefficients)

$`(Intercept)`
[1] 0.00000000000000005247539

$with_internet
[1] 0.3764094

$work_from_home
[1] 0.1495059

$sixtyfive_and_older
[1] 0.1413235

$number_providers
[1] -0.4186472

$mean_lowest_cost
[1] 0.268266

$mean_d_mbps
[1] 0.5489032

$mean_lat_ms
[1] -0.51717

In [20]:
model <- lm(
    index_data1.2$index ~ .,
    data = index_data1.2
)

In [24]:
as.list(vif(model))

Warning message in summary.lm(object, ...):
“essentially perfect fit: summary may be unreliable”


$with_internet
[1] 2.328952

$work_from_home
[1] 1.295396

$sixtyfive_and_older
[1] 1.913127

$number_providers
[1] 1.422172

$mean_lowest_cost
[1] 1.136557

$mean_d_mbps
[1] 3.771698

$mean_lat_ms
[1] 3.32024